# Refine txt2img Prompts with Human Feedback


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CarperAI/trlx/blob/main/examples/notebooks/trlx_simulacra.ipynb)


#### Optimize a gpt2-based txt2img prompt generator to produce aesthetic prompts using https://github.com/JD-P/simulacra-aesthetic-captions

Notebook by [@smellslikeml](https://github.com/smellslikeml)

---

Execute the cells below to install [TRLX](https://github.com/CarperAI/trlx) for a colab environment.

In [ ]:
!git clone https://github.com/CarperAI/trlx.git
!git config --global --add safe.directory /content/trlx && cd /content/trlx && pip install -e .

Cloning into 'trlx'...
remote: Enumerating objects: 4652, done.
remote: Counting objects: 100% (457/457), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 4652 (delta 299), reused 380 (delta 252), pack-reused 4195
Receiving objects: 100% (4652/4652), 46.06 MiB | 27.26 MiB/s, done.
Resolving deltas: 100% (2863/2863), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/trlx
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# uninstall scikit_learn + jax to avoid numpy issues
!pip uninstall -y scikit_learn jax

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
Found existing installation: jax 0.3.25
Uninstalling jax-0.3.25:
  Successfully uninstalled jax-0.3.25


In [ ]:
import os

# run within repo
os.chdir('/content/trlx')
print(os.getcwd())

/content/trlx


In [ ]:
import sqlite3
from urllib.request import urlretrieve

import trlx

url = "https://raw.githubusercontent.com/JD-P/simulacra-aesthetic-captions/main/sac_public_2022_06_29.sqlite"
dbpath = "sac_public_2022_06_29.sqlite"

if not os.path.exists(dbpath):
  print(f"fetching {dbpath}")
  urlretrieve(url, dbpath)

conn = sqlite3.connect(dbpath)
c = conn.cursor()
c.execute(
    "SELECT prompt, rating FROM ratings "
    "JOIN images ON images.id=ratings.iid "
    "JOIN generations ON images.gid=generations.id "
    "WHERE rating IS NOT NULL;"
)

prompts, ratings = tuple(map(list, zip(*c.fetchall())))

fetching sac_public_2022_06_29.sqlite


Trlx uses [wandb](https://wandb.ai/) to log results. Make sure to set up an account and use your token to authenticate when prompted after executing the cell below.

In [ ]:
trlx.train(
    "gpt2",
    samples=prompts,
    rewards=ratings,
    eval_prompts=["Hatsune Miku, Red Dress"] * 64,
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

losses/loss: 3.34, losses/loss_q: 0.27, losses/loss_v: 0.05, losses/loss_cql: 11.13, losses/loss_awac: 1.92: 100%|█████████▉| 999/1000 [17:36<00:01,  1.04s/it]

                                                  Evaluation #10                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                  ┃ output                                                                                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Hatsune Miku, Red Dress │ , pink, red hair, and blue eyes, cyberpunk, by Ross Tran, Tom Bagshaw, art by         │
│                         │ yoshitaka aman, trending on ArtStation, 4k hd, UHD, 4k                                │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ Hatsune Miku, Red Dress │ , Black, S-I, 4K wide, ArtStation, realistic                                          │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ Hatsune Miku, Red Dress │ , 4k UHD                                                                              │
└─────────────────────────┴───────────────────────────────────────────────────────────────────────────────────────┘

losses/loss: 3.47, losses/loss_q: 0.27, losses/loss_v: 0.04, losses/loss_cql: 11.21, losses/loss_awac: 2.04: 100%|██████████| 1000/1000 [17:48<00:00,  4.36s/it]

                                          Evaluation #11                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                  ┃ output                                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Hatsune Miku, Red Dress │ , pink, blue, white, blue, blue, blue, deep, deep, deep colors, deep │
├─────────────────────────┼──────────────────────────────────────────────────────────────────────┤
│ Hatsune Miku, Red Dress │ , 4k UHD, full color, realistic                                      │
├─────────────────────────┼──────────────────────────────────────────────────────────────────────┤
│ Hatsune Miku, Red Dress │ , blue, red, red                                                     │
└─────────────────────────┴──────────────────────────────────────────────────────────────────────┘

losses/loss: 3.47, losses/loss_q: 0.27, losses/loss_v: 0.04, losses/loss_cql: 11.21, losses/loss_awac: 2.04: 100%|██████████| 1000/1000 [17:58<00:00,  1.08s/it]
